In [1]:
import numpy as np
import cv2

#영역 검출

face_cascade = cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml')

eye_cascade = cv2.CascadeClassifier('data/haarcascade_eye.xml')

img = cv2.imread('img/people1.jpg') #people3 은 png
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray)

temp1 = 0
real_x = 0
real_y = 0
real_w = 0

temp2 = 500
real_ex = 0
real_ey = 0

for (x,y,w,h) in faces:
    
    if(w > temp1):
        temp1 = w
        real_x = x
        real_y = y
        real_w = w
        
    roi = gray[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi)
    
    for (ex,ey,ew,eh) in eyes:
        if(ex < temp2):
            temp2 = ex
            real_ex = ex
            real_ey = ey
        
        
img_show = img.copy()
img_show = cv2.resize(img_show, (0,0), fx=0.5, fy=0.5)
cv2.imshow('Basic', img_show)

print("real x :", real_x)
print("real_y :", real_y)
print("real_w :", real_w)
print("real_ex :", real_ex)
print("real_ey :", real_ey)



#눈 합성

a = [
    "img/add1.png",
    "img/add2.png",
    "img/add3.png",
    "img/add4.png",
    "img/add5.png",
    "img/add6.png",
    "img/add7.png",
    "img/add8.png"
]
i = 0

while i < len(a):
    key = cv2.waitKey(0) & 0xFF
    if key == 13:

        eye_size = 0

        eye1 = cv2.imread(a[i], cv2.IMREAD_UNCHANGED)
        ex_float = real_ex*1.7
        ex_int = int(ex_float)
        eye_size = (real_w) - (ex_int)
        print(ex_float)
        print(ex_int)
        print(eye_size)
        eye1 = cv2.resize(eye1, (eye_size, eye_size))
        back_img = img.copy()

        _, mask = cv2.threshold(eye1[:,:,3], 1, 255, cv2.THRESH_BINARY)
        mask_inv = cv2.bitwise_not(mask)

        eye1 = cv2.cvtColor(eye1, cv2.COLOR_BGRA2BGR)
        start_eye_x = real_x + real_ex -10 
        start_eye_y = real_y + 30          

        roi = back_img[start_eye_y:start_eye_y+eye_size, start_eye_x:start_eye_x+eye_size]

        print("start x:", start_eye_x)
        print("start y:", start_eye_y)

        masked_fg = cv2.bitwise_and(eye1, eye1, mask=mask)
        masked_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

        added = masked_fg + masked_bg
        back_img[start_eye_y:start_eye_y+eye_size, start_eye_x:start_eye_x+eye_size] = added

        back_img = cv2.resize(back_img, (0,0), fx=0.5, fy=0.5)
        cv2.imshow('result', back_img)
        
    if key == 27:
        third_page = back_img.copy()
        cv2.imwrite("img/third_page.jpg", third_page)
        cv2.destroyAllWindows()
        break        
    
    if(i < 7):
        i += 1
    else:
        i = 0
        

cv2.imshow('third_page', third_page)



#얼굴 보정

win_title = 'fourth_page'
half = 50              
isDragging = False


def liquify(img, cx1,cy1, cx2,cy2) :
    x, y, w, h = cx1-half, cy1-half, half*2, half*2
    
    roi = img[y:y+h, x:x+w].copy()
    out = roi.copy()

    offset_cx1,offset_cy1 = cx1-x, cy1-y
    offset_cx2,offset_cy2 = cx2-x, cy2-y
    
    tri1 = [[ (0,0), (w, 0), (offset_cx1, offset_cy1)], # 상,top
            [ [0,0], [0, h], [offset_cx1, offset_cy1]], # 좌,left
            [ [w, 0], [offset_cx1, offset_cy1], [w, h]], # 우, right
            [ [0, h], [offset_cx1, offset_cy1], [w, h]]] # 하, bottom

    tri2 = [[ [0,0], [w,0], [offset_cx2, offset_cy2]], # 상, top
            [ [0,0], [0, h], [offset_cx2, offset_cy2]], # 좌, left
            [ [w,0], [offset_cx2, offset_cy2], [w, h]], # 우, right
            [ [0,h], [offset_cx2, offset_cy2], [w, h]]] # 하, bottom

    
    for i in range(4):
        matrix = cv2.getAffineTransform( np.float32(tri1[i]), \
                                         np.float32(tri2[i]))
        warped = cv2.warpAffine( roi.copy(), matrix, (w, h), \
            None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)
        
        mask = np.zeros((h, w), dtype = np.uint8)
        cv2.fillConvexPoly(mask, np.int32(tri2[i]), (255,255,255))
        
        warped = cv2.bitwise_and(warped, warped, mask=mask)
        out = cv2.bitwise_and(out, out, mask=cv2.bitwise_not(mask))
        out = out + warped

    img[y:y+h, x:x+w] = out
    cv2.imshow(win_title, img)
    
    
    key = cv2.waitKey(0)
    
    if key == 13:
        for i in range(4):
            matrix = cv2.getAffineTransform( np.float32(tri2[i]), \
                                             np.float32(tri1[i]))
            warped = cv2.warpAffine( roi.copy(), matrix, (w, h), \
                None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)

            mask = np.zeros((h, w), dtype = np.uint8)
            cv2.fillConvexPoly(mask, np.int32(tri1[i]), (255,255,255))

            warped = cv2.bitwise_and(warped, warped, mask=mask)
            out = cv2.bitwise_and(out, out, mask=cv2.bitwise_not(mask))
            out = out + warped

        img[y:y+h, x:x+w] = out
        cv2.imshow(win_title, img)
        
    if key == 27:
        cv2.imwrite("img/result.jpg", img)
        cv2.destroyAllWindows()
    

def onMouse(event,x,y,flags,param):     
    global cx1, cy1, isDragging, img
    
    if event == cv2.EVENT_MOUSEMOVE:  
        if not isDragging :
            img_draw = img.copy()
            cv2.rectangle(img_draw, (x-half, y-half), \
                    (x+half, y+half), (0,255,0)) 
            
    elif event == cv2.EVENT_LBUTTONDOWN :   
        isDragging = True       
        cx1, cy1 = x, y 
    
    elif event == cv2.EVENT_LBUTTONUP :
        if isDragging:
            isDragging = False 
            liquify(img, cx1, cy1, x, y)
            
                
if __name__ == '__main__' :
    img = cv2.imread("img/third_page.jpg")
    cv2.putText(img, "HOW TO USE : Drag your mouse little by little", (10,30), 
               cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,0,0))
    h, w = img.shape[:2]

    cv2.namedWindow(win_title)
    cv2.setMouseCallback(win_title, onMouse)
    cv2.imshow(win_title, img)
    

        
cv2.waitKey(0)
cv2.destroyAllWindows()


real x : 221
real_y : 193
real_w : 313
real_ex : 59
real_ey : 83
100.3
100
213
start x: 270
start y: 223
100.3
100
213
start x: 270
start y: 223
100.3
100
213
start x: 270
start y: 223
100.3
100
213
start x: 270
start y: 223
